# Cars sale

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В распоряжении находятся исторические данные: технические характеристики, комплектации и цены автомобилей.  

#### Цель  
Необходимо построить модель для определения рыночной стоимости автомобиля.  

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

#### Описание данных  

**Признаки**  
DateCrawled — дата скачивания анкеты из базы  
VehicleType — тип автомобильного кузова  
RegistrationYear — год регистрации автомобиля  
Gearbox — тип коробки передач  
Power — мощность (л. с.)  
Model — модель автомобиля  
Kilometer — пробег (км)  
RegistrationMonth — месяц регистрации автомобиля  
FuelType — тип топлива  
Brand — марка автомобиля  
NotRepaired — была машина в ремонте или нет  
DateCreated — дата создания анкеты  
NumberOfPictures — количество фотографий автомобиля  
PostalCode — почтовый индекс владельца анкеты (пользователя)  
LastSeen — дата последней активности пользователя  
**Целевой признак**  
Price — цена (евро)  

# 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

Открываем данные

In [37]:
data_auto = pd.read_csv('/datasets/autos.csv')

In [38]:
data_auto.info()
data_auto.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [39]:
data_auto.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


В столбце **NumberOfPictures** все значения равны нулю, то есть такой столбец будет бесполезен при обучении, поэтому его стоит убрать. Также стоит убрать столбцы **DateCrawled**, **RegistrationMonth**, **DateCreated**, **LastSeen**, **PostalCode**, как невлияющие на цену.

In [40]:
data_auto.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'LastSeen', 'PostalCode'], axis = 1, inplace = True)

Приведем данные некоторых столбцов к типу **category**, так как это требуется для некоторых моделей

In [41]:
for feat in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']:
    data_auto[feat] = data_auto[feat].astype('category')

In [42]:
cat = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
data_auto[cat] = data_auto[cat].astype('category')

Посмотрим какой результат выдаст модель **LGBMRegressor** без обработки данных

In [43]:
data_target = data_auto['Price']
data_features = data_auto.drop(['Price'], axis = 1)

In [44]:
data_features_train, data_features_test, data_target_train, data_target_test = train_test_split(data_features, data_target, test_size=0.2, random_state=12345)

In [45]:
%%time
model = lgb.LGBMRegressor(num_threads = 2,  random_state = 12345)
model.fit(data_features_train, data_target_train)

CPU times: user 3.7 s, sys: 0 ns, total: 3.7 s
Wall time: 3.69 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, num_threads=2,
              objective=None, random_state=12345, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [46]:
%%time
pred = model.predict(data_features_test)

CPU times: user 822 ms, sys: 1.45 ms, total: 823 ms
Wall time: 814 ms


In [47]:
print('rmse =', mean_squared_error(data_target_test, pred)**0.5)

rmse = 1785.7374916790977


Без обработки данных среднеквадратическая ошибка составила 1785 евро. Сравним это значение со случаем, когда вместо предсказанных значений используется константа - среднее значение:

In [48]:
const_check = np.full((len(data_target_test), 1), data_target_test.mean())

In [49]:
print('rmse =', mean_squared_error(data_target_test, const_check)**0.5)

rmse = 4520.743927126966


Модель **LGBMRegressor** без предобработки данных показывает результат лучше, чем константная модель

### Предобработка

Посмотрим, какой процент пропусков в каждом столбце

In [50]:
(data_auto.isnull().sum() / data_auto.shape[0] * 100).sort_values(ascending = False)

NotRepaired         20.079070
VehicleType         10.579368
FuelType             9.282697
Gearbox              5.596709
Model                5.560588
Brand                0.000000
Kilometer            0.000000
Power                0.000000
RegistrationYear     0.000000
Price                0.000000
dtype: float64

#### столбцы RegistrationYear, Price и Power

Так как сервис занимается продажей автомобилей с пробегом, а не антикварных автомобилей, то поставим ограничимся временым промежутком с 1970 по 2020 (если предположить, что данные собирались и до настоящего времени), также будем рассматривать только те машины, цена которых больше 50 евро, а мощность больше 20 л.с.:

In [51]:
data_auto = data_auto.query("RegistrationYear > 1970 and RegistrationYear <= 2020 and Price > 50 and Power > 20")

#### столбец NotRepaired

В описании говорится, что этот столбец означает, была машина в ремонте или нет. Но остается непонятно, какой имеется ввиду ремонт: крупный, после аварии или текущий. Кроме того, получается, что если машина сломана и не ремонтировалась, то ей присваивается знаечение **yes** - не ремонтировалась.  
Получается, что признак сам по себе важный, но критерии присвоения значания непонятны. Поэтому пропущенные значения оставим в отдельной категории **nan**

In [52]:
def make_nan(feature):
    data_auto[feature] = data_auto[feature].astype('object')
    data_auto[feature].fillna('nan', inplace = True)
    data_auto[feature] = data_auto[feature].astype('category')

In [53]:
make_nan('NotRepaired')

In [54]:
data_auto['NotRepaired'].value_counts()

no     229119
nan     46532
yes     29381
Name: NotRepaired, dtype: int64

#### столбец VehicleType

В столбце **VehicleType** тоже пропуски заменим значением **nan**

In [55]:
make_nan('VehicleType')

#### столбец FuelType

In [56]:
data_auto['FuelType'].value_counts()

petrol      190654
gasoline     89295
lpg           4785
cng            497
hybrid         204
other           79
electric        40
Name: FuelType, dtype: int64

Заполним пропуски в столбце **FuelType** значением **gasoline**, так как бензиновый двигатель самый распространенный и можно с ожидать, что если клиент приложения не знает какой у него двигатель, то этот двигатель бензиновый, иначе клиент бы знал у него двигатель.  
Также заменим значение **petrol** на **gasoline**, потому что это одно и то же - безин

In [57]:
data_auto['FuelType'].fillna('gasoline', inplace = True)
data_auto['FuelType'] = data_auto['FuelType'].replace({'petrol' : 'gasoline'})
data_auto['FuelType'] = data_auto['FuelType'].astype('category')

#### стобец Model

В данном столбце пропуски просто удалим, потому что моделей у брендов много и в ценообразовании должны играть немалую роль, поэтому заполнять чем попало не хочется.

In [58]:
data_auto.dropna(subset = ['Model'], inplace = True)

#### столбец Gearbox

In [59]:
data_auto['Gearbox'].value_counts()

manual    230287
auto       57631
Name: Gearbox, dtype: int64

Заполним пропуски значением **manual**, так как оно встречается гораздо чаще значения **auto**

In [60]:
data_auto['Gearbox'].fillna('manual', inplace = True)

### Вывод

In [27]:
data_auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293091 entries, 2 to 354368
Data columns (total 10 columns):
Price               293091 non-null int64
VehicleType         293091 non-null category
RegistrationYear    293091 non-null int64
Gearbox             293091 non-null category
Power               293091 non-null int64
Model               293091 non-null category
Kilometer           293091 non-null int64
FuelType            293091 non-null category
Brand               293091 non-null category
NotRepaired         293091 non-null category
dtypes: category(6), int64(4)
memory usage: 13.2 MB


Пропуски заполнены или удалены

# 2. Обучение моделей

### Модель LightGBM

Разделим данные на признаки и целевой признак

In [38]:
data_target = data_auto['Price']
data_features = data_auto.drop(['Price'], axis = 1)

Разделим данные на тренировочную и тестовую выборки

In [39]:
data_features_train, data_features_test, data_target_train, data_target_test = train_test_split(data_features, data_target, test_size=0.2, random_state=12345)

Задаем параметры модели

In [59]:
my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [60]:
param_grid = {'max_depth': [20, 30], 'num_leaves': [501, 701], 'subsample_for_bin': [50000, 100000, 150000, 200000]}

In [61]:
model_grid = GridSearchCV(lgb.LGBMRegressor(num_threads = 2,  random_state = 12345), param_grid = param_grid, scoring=my_scorer)

Обучим модель

In [62]:
%%time
model_grid.fit(data_features_train, data_target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 8min 57s, sys: 13 s, total: 9min 10s
Wall time: 9min 10s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     num_threads=2, objective=None,
                                     random_state=12345, reg_...0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [20, 30], 'num_leaves': [501, 701],
                         'subsample_for_b

Лучшие параметры и среднеквадратичная ошибка следующие:

In [63]:
print('best_params:', model_grid.best_params_)
print('rmse =', (abs(model_grid.best_score_))**0.5)

best_params: {'max_depth': 30, 'num_leaves': 701, 'subsample_for_bin': 100000}
rmse = 1505.8756422194226


Посмотрим, что будет на тестовой выборке

In [64]:
%%time
pred = model_grid.predict(data_features_test)

CPU times: user 2.78 s, sys: 3.97 ms, total: 2.78 s
Wall time: 2.8 s


In [65]:
print('rmse =', mean_squared_error(data_target_test, pred)**0.5)

rmse = 1477.6040865794737


Среднеквадратичная ошибка составила 1478

### Модель CatBoost

Рассмотрим теперь модель **CatBoostRegressor**

Задаем параметры модели

In [67]:
param_grid_cat = {'iterations': [200], 'depth': [15], 'learning_rate': [0.5]}

In [68]:
model_grid_cat = GridSearchCV(CatBoostRegressor(random_state = 12345), param_grid = param_grid_cat, cv=3)

In [40]:
cat_features = ['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'NotRepaired']

Обучим модель

In [70]:
%%time
model_grid_cat.fit(data_features_train, data_target_train, cat_features=cat_features, verbose=10)

0:	learn: 2995.5824025	total: 3.89s	remaining: 12m 54s
10:	learn: 1559.4839489	total: 36.3s	remaining: 10m 22s
20:	learn: 1453.0042643	total: 1m 9s	remaining: 9m 48s
30:	learn: 1373.8656369	total: 1m 41s	remaining: 9m 12s
40:	learn: 1313.1247359	total: 2m 17s	remaining: 8m 52s
50:	learn: 1267.7973395	total: 2m 54s	remaining: 8m 28s
60:	learn: 1239.1986986	total: 3m 31s	remaining: 8m 2s
70:	learn: 1206.7036669	total: 4m 10s	remaining: 7m 34s
80:	learn: 1181.5891782	total: 4m 49s	remaining: 7m 5s
90:	learn: 1162.0620986	total: 5m 28s	remaining: 6m 33s
100:	learn: 1145.6777969	total: 6m 8s	remaining: 6m
110:	learn: 1128.5246113	total: 6m 48s	remaining: 5m 27s
120:	learn: 1108.1955596	total: 7m 27s	remaining: 4m 52s
130:	learn: 1093.1762158	total: 8m 4s	remaining: 4m 15s
140:	learn: 1083.1607025	total: 8m 42s	remaining: 3m 38s
150:	learn: 1069.8597774	total: 9m 19s	remaining: 3m 1s
160:	learn: 1056.9464547	total: 9m 56s	remaining: 2m 24s
170:	learn: 1045.0207402	total: 10m 35s	remaining: 1

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostRegressor object at 0x7fb3a2b1d110>,
             iid='warn', n_jobs=None,
             param_grid={'depth': [15], 'iterations': [200],
                         'learning_rate': [0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [71]:
predict_cat = model_grid_cat.predict(data_features_test)

In [72]:
print('rmse =', mean_squared_error(data_target_test, predict_cat)**0.5)

rmse = 1534.2342434655181


Значение среднеквадратичной ошибки на тестовой выборке составило 1534

### Модель RandomForestRegressor

In [61]:
le = LabelEncoder()

In [62]:
for feat in ['VehicleType', 'Gearbox', 'Model','FuelType', 'Brand', 'NotRepaired']:
    le.fit(data_auto[feat])
    data_auto[feat + '_le'] = le.transform(data_auto[feat])

In [64]:
target_le = data_auto['Price']
features_le = data_auto.drop(['Price','VehicleType', 'Gearbox', 'Model','FuelType', 'Brand', 'NotRepaired'], axis = 1)

In [65]:
features_train_le, features_test_le, target_train_le, target_test_le = train_test_split(features_le, target_le, test_size=0.2, random_state=12345)

In [66]:
model_forest = RandomForestRegressor(n_jobs = -1, random_state = 12345)

In [67]:
param_grid_forest = {'n_estimators': [50, 100], 'max_depth': [10, 15, 20]}

In [68]:
model_grid_forest = GridSearchCV(model_forest, param_grid = param_grid_forest, cv=3)

In [69]:
%%time
model_grid_forest.fit(features_train_le, target_train_le)

CPU times: user 8min 48s, sys: 3.03 s, total: 8min 51s
Wall time: 8min 59s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=-1,
                                             oob_score=False,
                                             random_state=12345, verbose=0,
                                             warm_start=False),
             iid='warn', n_jobs=None,
            

In [71]:
print('best_params:', model_grid_forest.best_params_)

best_params: {'max_depth': 20, 'n_estimators': 100}


In [72]:
forest_predict = model_grid_forest.predict(features_test_le)

In [73]:
print('rmse =', mean_squared_error(target_test_le, forest_predict)**0.5)

rmse = 1564.8396139296783


Значение среднеквадратичной ошибки составило 1565 евро

# 3. Анализ моделей

#### модель LGBMRegressor

In [75]:
model_lgb = lgb.LGBMRegressor(num_threads = 2,  random_state = 12345, max_depth = 30, num_leaves = 701, subsample_for_bin = 100000)

In [76]:
%%time
model_lgb.fit(data_features_train, data_target_train)

CPU times: user 10.6 s, sys: 304 ms, total: 10.9 s
Wall time: 10.9 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=30,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=701, num_threads=2,
              objective=None, random_state=12345, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=100000,
              subsample_freq=0)

In [80]:
%%time
predict_lgb = model_lgb.predict(data_features_test)

CPU times: user 2.58 s, sys: 0 ns, total: 2.58 s
Wall time: 2.56 s


In [82]:
print('rmse =', mean_squared_error(data_target_test, predict_lgb)**0.5)

rmse = 1477.6040865794737


Модель **LGBMRegressor** обучается за 11 секунд, делает предсказание за 2.58 секунд, а среднеквадратичная ошибка составляет 1477 евро.

#### модель CatBoost

In [41]:
model_cat = CatBoostRegressor(iterations = 200, depth = 15, learning_rate = 0.5, random_state = 12345)

In [42]:
%%time
model_cat.fit(data_features_train, data_target_train, cat_features=cat_features, verbose=50)

0:	learn: 2966.4442979	total: 4.51s	remaining: 14m 57s
50:	learn: 1309.8248158	total: 3m 38s	remaining: 10m 39s
100:	learn: 1199.9876319	total: 7m 32s	remaining: 7m 23s
150:	learn: 1131.5944799	total: 11m 27s	remaining: 3m 43s
199:	learn: 1088.8563911	total: 15m 19s	remaining: 0us
CPU times: user 14min 26s, sys: 56.2 s, total: 15min 23s
Wall time: 15min 24s


In [43]:
%%time
predict_cat = model_cat.predict(data_features_test)

CPU times: user 882 ms, sys: 48.5 ms, total: 931 ms
Wall time: 921 ms


In [44]:
print('rmse =', mean_squared_error(data_target_test, predict_cat)**0.5)

rmse = 1534.2342434655181


Модель **CatBoost** обучается за 15 минут, делает предсказание за 0.882 секунд, а среднеквадратичная ошибка составляет 1534 евро.

#### модель RandomForestRegressor

In [74]:
model_forest = RandomForestRegressor(n_jobs = -1, random_state = 12345, n_estimators = 100,max_depth = 20)

In [75]:
%%time
model_forest.fit(features_train_le, target_train_le)

CPU times: user 1min 3s, sys: 532 ms, total: 1min 3s
Wall time: 1min 5s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=12345, verbose=0,
                      warm_start=False)

In [76]:
%%time
predict_forest = model_forest.predict(features_test_le)

CPU times: user 2.19 s, sys: 0 ns, total: 2.19 s
Wall time: 2.19 s


In [77]:
print('rmse =', mean_squared_error(target_test_le, predict_forest)**0.5)

rmse = 1564.8396139296783


Модель **RandomForestRegressor** обучается за 8 минут, делает предсказание 2.19 секунд, а среднеквадратичная ошибка составляет 1580 евро.

# Вывод

Лучше всего себя показала модель LGBMRegressor, которая обучается за 11 секунд: лучше, чем 1 и 10 минут у остальных. Также у этой модели среднеквадратичная ошибка меньше, чем у остальных моделей и составляет 1477 евро. На предсказание затрачивается 2.58 секунд, что является приемленым результатом.